In [2]:
import pandas as pd
import numpy as np
import cv2

In [61]:
img = cv2.imread("/home/sabrine/notebook/framesRGB/1493821823811310160.jpg")

In [62]:
img.shape

(540, 960, 3)

In [63]:
cv2.imshow('', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
('name', 'Wissem')
('distance', 0.9801111671584597)
[406, 55]
('name', 'Laurene')
('distance', 1.2919695541530991)
[310, 126]

In [64]:
laurene=(310, 126)
#wissem=(471, 77)
wissem=(406, 55)

In [70]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [75]:
test = cv2.putText(img, "This one!", laurene, font, 0.2, (0, 255, 0), 2, cv2.LINE_4)

In [76]:
cv2.imshow('', test)
cv2.waitKey()
cv2.destroyAllWindows()

In [65]:
rectangle1 = cv2.circle(img,laurene,20,(0,0,255),3) # red
rectangle2 = cv2.circle(rectangle1,wissem,20,(0,255,0),3) # green
#rectangle3 = cv2.circle(rectangle2,sabrine,20,(255,0,0),3) # blue

In [67]:
cv2.imshow('', rectangle2)
cv2.waitKey()
cv2.destroyAllWindows()